<h1><div align='center'><font size='6' color='brick'> Data Preprocessing for Modeling</font></div></h1>
<br>

**This document will cover:**

- Removing and imputing missing values from the dataset 
- Getting categorical data into shape for machine learning algorithms 
- Partitioning a dataset into separate training and test sets 
- Bringing features onto the same scale

At this stage, all we have is raw data with a **number of variables/fields(columns)** and **observations(rows)**. We are yet to do feature engineering at which stage we will refer to the variables as features which will form the inputs to future models. As of now, the columns are referred to as variables.

## Importing packages

In [48]:
import pandas as pd 
from io import StringIO 
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

## Dealing with missing data

Real-world data sets are often missing a large number of values. This may be due to the nature of the data collected, errors in the data collection process or certain fields are not applicable.

In [2]:
# Creating a csv to work with
csv_data =\
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


### Finding missing data

#### isnull( )
The `isnull` method returns a DataFrame with Boolean values that indicate whether a cell contains a numeric value (False) or if data is missing (True).


In [3]:
df.isnull()

,A,B,C,D
0,False,False,False,False
1,False,False,True,False
2,False,False,False,True


`.notnull()` is the negation of the above.

In [4]:
df.notnull()

,A,B,C,D
0,True,True,True,True
1,True,True,False,True
2,True,True,True,False


#### Missing values per column
Using the `sum()` method, we can then return the number of missing values per column as follows:

In [5]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

### Eliminating observations or variables with missing values

This drops observations with missing values.

In [6]:
 df.dropna(axis=0) # axis=0 is the default

,A,B,C,D
0,1.0,2.0,3.0,4.0


To drop columns with missing values:

In [7]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [8]:
# only drop rows where all fields are NaN
# (returns the whole array here since we don't 
# have a row with where all values are NaN 
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [9]:
 # drop rows that have less than 4 real values
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [10]:
# only drop rows where NaN appear in specific columns (here: 'C')
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


### Imputing missing values

Eliminating observations or variables with missing values can be dangerous while building models. For example, in fraud detection models, the missing value could most likely be a case of fraud which is the reason it's an anomaly in the first place. 

In order to avoid errors and biases that can arise from eliminating data points with missing values, we can replace the missing values with a relevant/representative value imputed from the the other values of that variable.

#### Replacing with column mean

The SimpleImputer simply replaces the missing value with the mean value of the entire feature column. 

In [11]:
imp = SimpleImputer(missing_values = np.nan, strategy = 'mean') 
imp = imp.fit(df.values)
imputed_data = imp.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [12]:
imp

SimpleImputer()

**Other options for the strategy parameter are:**
- _median_ 
- _constant_: When strategy == “constant”, there is another parameter named fill_value, which is used to replace all occurrences of missing_values. If left to the default, fill_value will be 0 when imputing numerical data and “missing_value” for strings or object data types.
- _most_frequent_: This replaces the missing values with the most frequent values. 
This is useful for imputing **categorical variables**, for example, a feature column that stores an encoding of color names, such as red, green, and blue


**For numerical variables:**
- Right off the bat, you can tell why this is dangerous. Means are notorious for being affected by outliers. Replacing NaNs by the column mean could further bias the data.

- Another option is to replace by the median, but there are still better ways to deal with missing values in numerical variables.

#### KNN Imputer

The KNNImputer class provides imputation for filling in missing values using the k-Nearest Neighbors approach.

By default, a euclidean distance metric that supports missing values, nan_euclidean_distances, is used to find the nearest neighbors. Each observation’s missing values are imputed using the mean values from n_neighbors nearest neighbors found in the training data sets. Two observations are close if the features that neither is missing are close.

In [13]:
X = [[1, 2, np.nan], [3, 4, 3], [np.nan, np.nan, 5], [8, 8, 7]] 
df = pd.DataFrame(X, columns=['A', 'B', 'C'])
df

,A,B,C
0,1.0,2.0,NaN
1,3.0,4.0,3.0
2,NaN,NaN,5.0
3,8.0,8.0,7.0


* We set n_neighbors to 2, which means that the number of neighboring observations to use for imputation is 2. **n_neighbors’ default value is 5.**


* We also set weights to “uniform”, which means that all points in each neighborhood are weighted equally. Besides, weights can be also set to “distance”, which means that we weight points by the inverse of their distance. 
In other word, closer neighbors of a query point will have a greater influence than neighbors which are further away. **weights’ default value is “uniform”**

In [14]:
imputer = KNNImputer(n_neighbors=2, weights="uniform") 
imputer.fit_transform(X) # you will get a np array

array([[1. , 2. , 5. ],
       [3. , 4. , 3. ],
       [5.5, 6. , 5. ],
       [8. , 8. , 7. ]])

#### Processing training and testing data

In [15]:
data_train = {'A': [3, 2, np.nan, 4, 3],\
              'B': [3, np.nan, 4, 4, 5],\
              'C':[np.nan, 4.8, 5.1, 4.9, 5.2],\
              'D':[6, 7, 9, np.nan, 10]}
df2_train = pd.DataFrame(data = data_train)
df2_train

,A,B,C,D
0,3.0,3.0,NaN,6.0
1,2.0,NaN,4.8,7.0
2,NaN,4.0,5.1,9.0
3,4.0,4.0,4.9,NaN
4,3.0,5.0,5.2,10.0


Pandas' `mean` function ignores NaNs while calculating the mean. It has an argument `skipna` set to True by default. Also,

- `axis=0` gives column-wise mean
- `axis=1` gives row-wise mean

In [16]:
df2_train.mean()

A    3.0
B    4.0
C    5.0
D    8.0
dtype: float64

In [17]:
imp2 = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imped = imp2.fit(df2_train)
imputed_df2_train = imped.transform(df2_train) 
# old version need "df2_train" to be "df2_train.values" 
imputed_df2_train

array([[ 3. ,  3. ,  5. ,  6. ],
       [ 2. ,  4. ,  4.8,  7. ],
       [ 3. ,  4. ,  5.1,  9. ],
       [ 4. ,  4. ,  4.9,  8. ],
       [ 3. ,  5. ,  5.2, 10. ]])

Our test data df2_test is as follows:

In [18]:
data_test = {'A': [2, np.nan, 3], 'B': [4, 5, np.nan],\
             'C':[np.nan, 4, 5], 'D':[7, 8, np.nan]}
df2_test = pd.DataFrame(data = data_test)
df2_test

,A,B,C,D
0,2.0,4.0,NaN,7.0
1,NaN,5.0,4.0,8.0
2,3.0,NaN,5.0,NaN


In [19]:
df2_test.mean(axis = 0)

A    2.5
B    4.5
C    4.5
D    7.5
dtype: float64

In [20]:
imputed_df2_test = imp2.transform(df2_test)
imputed_df2_test

array([[2., 4., 5., 7.],
       [3., 5., 4., 8.],
       [3., 4., 5., 8.]])

## Handling categorical data

In [21]:
df=pd.DataFrame([['green', 'M', 10.1, 'class1'],
                   ['red', 'L', 13.5, 'class2'],
                   ['blue', 'XL', 15.3, 'class1']])
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


The newly created DataFrame contains a nominal feature (color), an ordinal feature (size), and a numerical feature (price) column. The class labels (assuming that we created a dataset for a supervised learning task) are stored in the last column.

### Mapping Ordinal variables

To make sure that the learning algorithm interprets the ordinal features correctly, we need to convert the categorical string values into integers. Unfortunately, there is no convenient function that can automatically derive the correct order of the labels of our size feature, so we have to define the mapping manually. In the following simple example, let’s assume that we know the numerical difference between features, for example, XL = L + 1 = M + 2:

In [22]:
# we define the mapping manually using the dictionary
size_mapping = {'XL': 3,
                'L': 2,
                'M': 1}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


To reverse the mapping later:

In [23]:
inv_size_mapping = {v: k for k, v in size_mapping.items()}
inv_size_mapping

{3: 'XL', 2: 'L', 1: 'M'}

### Encoding class labels

We need to remember that class labels are not ordinal, and it doesn’t matter which integer number we assign to a particular string label. Thus, we can simply enumerate the class labels, starting at 0.

There is a convenient LabelEncoder class directly implemented in scikit-learn to achieve this. 

REMEMBER: the fit_transform method is just a shortcut for calling fit and transform separately


In [25]:
# Label encoding with sklearn's LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 0])

In [26]:
 df['classlabel'] = y
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


We can use the inverse_transform method to transform the integer class labels back into their original string representation:

In [27]:
df['classlabel'] = class_le.inverse_transform(y)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


### One-hot encoding on nominal variables

In [28]:
X = df[['color', 'size', 'price']].values
X

array([['green', 1, 10.1],
       ['red', 2, 13.5],
       ['blue', 3, 15.3]], dtype=object)

In [30]:
type(X)

numpy.ndarray

In [31]:
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

Although the color values don’t come in any particular order, a learning algorithm will now assume that **green is larger than blue, and red is larger than green.**

A common workaround for this problem is to use a technique called **one-hot encoding**. The idea behind this approach is to create a new dummy feature for each unique value in the nominal feature column. Here, we would convert the color feature into three new features: blue, green, and red. Binary values can then be used to indicate the particular color of a sample; for example, a blue t-shirt can be encoded as blue=1, green=0, red=0. 

#### Scikit-learn
To perform this transformation, we can use the OneHotEncoder that is implemented in the scikit-learn.preprocessing module.

In [33]:
X = df[['color', 'size', 'price']].values
ct = ColumnTransformer([("color",OneHotEncoder(), [0])], remainder ='passthrough') 
# the "color" parameter seems to be redundant, but it must exist when using this package 
X = ct.fit_transform(X)
X

array([[0.0, 1.0, 0.0, 1, 10.1],
       [0.0, 0.0, 1.0, 2, 13.5],
       [1.0, 0.0, 0.0, 3, 15.3]], dtype=object)

#### pd.get_dummies

An even more convenient way to create those dummy features via one-hot encoding is to use the get_dummies method implemented in pandas. Applied to a DataFrame, the get_dummies method will only convert string columns and leave all other columns unchanged:

In [35]:
# one-hot encoding via pandas
pd.get_dummies(df[['price', 'color', 'size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


#### Dealing with Multi-collinearity
When we are using one-hot encoding datasets, we have to keep in mind that it introduces multi- collinearity, which can be an issue for certain methods (for instance, methods that require matrix inversion). If features are highly correlated, matrices are computationally difficult to invert, which can lead to numerically unstable estimates. To reduce the correlation among variables, we can simply remove one feature column from the one-hot encoded array. Note that we do not lose any important information by removing a feature column, though; for example, if we remove the column color_blue, the feature information is still preserved since if we observe color_green=0 and color_red=0, it implies that the observation must be blue. For a nominal variable with K categories, we only need K-1 dummies.

We can drop the first column by using the `drop_first` parameter.

In [36]:
 # multicollinearity guard in get_dummies
pd.get_dummies(df[['price', 'color', 'size']], drop_first=True)


,price,size,color_green,color_red
0,10.1,1,1,0
1,13.5,2,0,1
2,15.3,3,0,0


If using the OneHotEncoder:

In [37]:
X=df[['color', 'size', 'price']].values
ct = ColumnTransformer([("color", OneHotEncoder(drop = 'first'), [0])],\
                       remainder = 'passthrough') 
X = ct.fit_transform(X)
X

array([[1.0, 0.0, 1, 10.1],
       [0.0, 1.0, 2, 13.5],
       [0.0, 0.0, 3, 15.3]], dtype=object)

## Partitioning a dataset into separate training and test sets

I will use the Wine dataset.

Refer: https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.names

In [39]:
 df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)

# if the Wine dataset is temporarily unavailable from the
# UCI machine learning repository, un-comment the following line
# of code to load the dataset from a local path:
# df_wine = pd.read_csv('wine.data', header=None)
df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alkalinity of ash', 'Magnesium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
                   'Proline']
print('Class labels', np.unique(df_wine['Class label']))
df_wine.head()

Class labels [1 2 3]


,Class label,Alcohol,Malic acid,Ash,Alkalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [41]:
# X contains the features
# y contains the class label 
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

#Next, we split into train and test data sets
X_train, X_test, y_train, y_test =\
    train_test_split(X, y,
                     test_size=0.3,
                     random_state=0,
                     stratify=y)

**Note:**
- Default value for `test_size` is .25
- `stratify` ensures a representative proportion of each class are distributed across the test and train data


## Feature scaling

There are two common approaches to bring different features onto the same scale: normalization and standardization.

### Normalization
Most often, normalization refers to the rescaling of the features to a range of [0, 1], which is a special case of min-max scaling. To normalize our data, we can simply apply the min-max scaling to each feature column, where the new value $ x_{norm}^{(i)} $ of a sample x(i) can be calculated as follows:

$$ x_{norm}^{(i)} = \frac{x^{(i)}-x_{min}}{x_{max}-x_{min}} $$

In [43]:
mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

In [44]:
ex_train = np.array([3, 4, 5, 6, 7, 8]).reshape(6, 1)
ex_train

array([[3],
       [4],
       [5],
       [6],
       [7],
       [8]])

In [45]:
 mms_ex = MinMaxScaler()
ex_train_norm = mms_ex.fit_transform(ex_train)
ex_train_norm

array([[0. ],
       [0.2],
       [0.4],
       [0.6],
       [0.8],
       [1. ]])

In [46]:
 ex_test = np.array([2, 3, 4, 5]).reshape(4, 1)
ex_test

array([[2],
       [3],
       [4],
       [5]])

In [47]:
 ex_test_norm = mms_ex.transform(ex_test)
ex_test_norm

array([[-0.2],
       [ 0. ],
       [ 0.2],
       [ 0.4]])

### Standardization
Using standardization, we center the feature columns at mean 0 with standard deviation 1.

$$z_{i}=\frac{x_i-\mu}{\sigma}$$

In [49]:
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)